<a href="https://colab.research.google.com/github/Kumarvels/GenAIProjects/blob/main/CTFAgent_Pentesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Paper CTFAgent: Penetration Testing AUtomation**

The paper titled "Measuring and Augmenting Large Language Models for Solving Capture-the-Flag Challenges" (arXiv:2506.17644), addresses a significant cybersecurity problem: the inability of large language models (LLMs) to effectively apply their technical knowledge to solve complex, dynamic, and interactive cybersecurity challenges, such as Capture-the-Flag (CTF) competitions.

These competitions simulate real-world hacking scenarios where participants must identify vulnerabilities, exploit them, and retrieve hidden "flags" (unique text strings) to demonstrate their skills. The paper highlights that while LLMs possess vast theoretical knowledge, they struggle to translate this into practical, actionable solutions in such environments, particularly due to limitations in contextual reasoning and interaction with live systems.



In [ ]:
# @title CTFAgent Prototype in Google Colab

# Step 1: Install required libraries and setup Ollama
# This will install all necessary Python packages.
!pip install langchain transformers sentence-transformers chromadb ollama langchain-community -q

# Download and install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# Start the Ollama server in the background.
import subprocess
import time
import os

# It's recommended to run Ollama on a GPU instance in Colab for better performance.
# Make sure to change the runtime type to include a GPU.
command = "ollama serve"
process = subprocess.Popen(command.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)

print("Starting Ollama server...")
# Give the server a moment to start up
time.sleep(5)

# Pull the required model. Here we use llama3.
!ollama pull llama3
print("Ollama server and model are ready.")


# Step 2: Import necessary libraries
from langchain_community.llms import Ollama
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

# Step 3: Set up the RAG Knowledge Base
# Sample cybersecurity knowledge (simplified vulnerability data)
knowledge_base = """
Buffer Overflow: Occurs when a program attempts to write more data to a buffer than the buffer can hold. This can overwrite adjacent memory, leading to a crash or execution of arbitrary code. A common exploit involves overwriting the return address on the stack with the address of malicious shellcode.
SQL Injection: A code injection technique that might destroy your database. SQL Injection is one of the most common web hacking techniques. It is the placement of malicious code in SQL statements, via web page input. A common exploit to bypass authentication is to use ' OR 1=1 --.
"""

# Split the knowledge base into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_text(knowledge_base)

# Create embeddings for the text chunks
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create a Chroma vector store from the texts and embeddings
vectorstore = Chroma.from_texts(texts, embeddings)

# Initialize the LLM (using the running Ollama instance)
llm = Ollama(model="llama3")

# Create a RetrievalQA chain for vulnerability identification
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

# Step 4: Define Agent Functions

def generate_exploit(vulnerability):
    """Generates an exploit for a given vulnerability using the RAG chain."""
    query = f"How can I exploit a {vulnerability} vulnerability?"
    result = qa_chain({"query": query})
    return result['result']

def simulate_command_execution(command):
    """Simulates the execution of a command in a sandboxed environment."""
    print(f"\nExecuting (simulated): {command}")
    # In a real-world scenario, this would interact with a secure, sandboxed environment.
    if "buffer overflow" in command.lower():
        return "Flag retrieved: FLAG{BufferOverflowSuccess_1a2b3c}"
    elif "sql injection" in command.lower() or "' or 1=1 --" in command.lower():
        return "Flag retrieved: FLAG{SQLInjectionSuccess_4d5e6f}"
    else:
        return "No flag retrieved. The command was either invalid or did not succeed."

# Step 5: Main Testing Loop

def run_ctfagent_challenge(challenge_description):
    """Runs the full CTF agent workflow for a given challenge."""
    print("--- Starting CTF Challenge ---")
    print(f"Challenge: {challenge_description}")

    # RAG-Understanding: Identify the vulnerability from the challenge description.
    vulnerability_result = qa_chain({"query": challenge_description})
    vulnerability = vulnerability_result['result']
    print(f"\nIdentified Vulnerability: {vulnerability}")

    # RAG-Exploiting: Generate a potential exploit for the identified vulnerability.
    exploit_suggestion = generate_exploit(vulnerability.split(":")[0]) # Extract the vulnerability type
    print(f"\nGenerated Exploit: {exploit_suggestion}")

    # Interactive Augmentation: Execute and retrieve flag.
    # In a real-world scenario, the agent would choose and execute commands based on the exploit_suggestion.
    # Here, we simulate user input for the command.
    command = input("Enter simulated command to execute exploit: ")
    flag = simulate_command_execution(command)
    print(f"Result: {flag}")

# Example Challenge
challenge = "Analyze the following network service for vulnerabilities: A web application with a login form."
run_ctfagent_challenge(challenge)